In [ ]:
!git clone https://github.com/khalld/deep-learning.git
!mv deep-learning dl
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
try:
    import pytorch_lightning as pl
except ModuleNotFoundError:
    !pip install --quiet pytorch-lightning>=1.4
try:
    import faiss
except ModuleNotFoundError:
    !pip install faiss-gpu

In [ ]:
import os
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import transforms
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from libs.code import *
from pytorch_lightning.loggers import TensorBoardLogger
import torch
from torchvision import transforms
import pytorch_lightning as pl

from libs.Dataset import *
from libs.util import *
from libs.SiameseNetwork import TripletNetworkTask
# non necessari !
# from libs.code import *
# from libs.VAE import *

Imposto i seed e le variabili globali:

In [ ]:
random.seed(1996)
np.random.seed(1996)
pl.seed_everything(1996)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

In [ ]:
PATH_DST = 'dataset/all_labels.csv'
PATH_GDRIVE = ''
NUM_WORKERS = 8
BATCH_SIZE = 32
NUM_EPOCHS = 4
GPUS = 0
PRETRAINED_MODEL_PATH =  'models/squeezeNet_pretrained.pth'
num_class = 3

Carico il dataset singolo

In [ ]:
# valori pretrained
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225] 

transforms = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(), transforms.Normalize(mean=mean, std=std)])

dst = TrashbinDataset(csv=PATH_DST, transform=transforms)

dst_train, dst_test = split_into_train_and_test(dst)

dst_train_loader = DataLoader(dst_train, num_workers=NUM_WORKERS, batch_size=BATCH_SIZE, shuffle=True)
dst_test_loader = DataLoader(dst_test, num_workers=NUM_WORKERS, batch_size=BATCH_SIZE, shuffle=False)

Estraggo le rappresentazioni rgb dai loader:

In [ ]:
dst_train_rep_rgb, dst_train_labels = extract_rgb_representations(loader=dst_train_loader)
dst_test_rep_rgb, dst_test_labels = extract_rgb_representations(loader=dst_test_loader)

Rappresentazioni di training:

In [ ]:
dst_train_rep_rgb.shape

Ottengo le predizioni sul test-set usando `predict_nn`:

In [ ]:
pred_test_label_rgb = predict_nn(dst_train_rep_rgb, dst_test_rep_rgb, dst_train_labels)
print(f"Sample di label: {pred_test_label_rgb}")

Valuto le performance della baseline

In [ ]:
classification_error = evaluate_classification(pred_test_label_rgb, dst_test_labels)
print(f"Classification error: {classification_error:0.2f}")

Importo per effettuare il training della triplenet il miglior modello trovato nella precedente relazione: `SqueezeNet v1.0`. Importo dunque i pesi già trovati dopo il training di 100 epoche .... <b>TODO migliora la descrizione</b> ... importo i pesi.. faccio le opportune modifiche ...

In [ ]:
# scarico il modello da pytorch
squeezeNet_1_0 = torch.hub.load('pytorch/vision:v0.10.0', 'squeezenet1_0', pretrained=True)
# applico le opportune modifiche
squeezeNet_1_0.classifier[1] = nn.Conv2d(512, num_class, kernel_size=(1,1), stride=(1,1))
squeezeNet_1_0.num_classes = num_class
# carico i pesi salvati
squeezeNet_1_0.load_state_dict(torch.load(PRETRAINED_MODEL_PATH))

# riduco l'ultimo modello alla funzione attività:
squeezeNet_1_0.classifier = nn.Identity()
squeezeNet_1_0(torch.zeros(1, 3, 224,224)).shape

In [ ]:
# uso il modello, allenato nel precedente progetto, per estrarre le rappresentazioni dal training e dal test set
dst_train_rep, dst_train_labels = extract_rep_squeezeNet(squeezeNet_1_0, dst_train_loader)
dst_test_rep, dst_test_labels = extract_rep_squeezeNet(squeezeNet_1_0, dst_test_loader)

Valuto le performance del sistema:

In [ ]:
# valuto le performance del sistema con rappresentazioni non ancora ottimizzate
pred_test_label = predict_nn(dst_train_rep, dst_test_rep, dst_train_labels)
classification_error = evaluate_classification(pred_test_label, dst_test_labels)
print(f"Classification error: {classification_error}")

Carico il dataset in triplette:

In [ ]:
dst_triplet = TripletTrashbin(root=PATH_DST, transform=transforms)

dst_train_triplet, dst_test_triplet = split_into_train_and_test(dst_triplet)

triplet_dataset_train_loader = DataLoader(dst_train_triplet, num_workers=NUM_WORKERS, batch_size=BATCH_SIZE, shuffle=True)
triplet_dataset_test_loader = DataLoader(dst_test_triplet, num_workers=NUM_WORKERS, batch_size=BATCH_SIZE)

In [ ]:
# TODO: mostra le immagini delle triplette

Alleno la rete con lr=0.002 che è il migliore trovato per SqueezeNet nel precedente progetto.

In [ ]:
triplet_trashbin_task =  TripletNetworkTask(squeezeNet_1_0, lr=0.002)
logger = TensorBoardLogger("metric_logs", name="test_trashbin_v1",
                            callbacks=[
                                ModelCheckpoint(save_weights_only=True, every_n_epochs=2),    # default save checkpoint every 10 times
                                LearningRateMonitor("epoch")
                                ], )

# TODO: salva ogni ...
# TODO: CALLBACK!!!!!
trainer = pl.Trainer(gpus=GPUS, logger = logger, max_epochs = 10, check_val_every_n_epoch=5, )
trainer.fit(triplet_trashbin_task, triplet_dataset_train_loader, triplet_dataset_test_loader)

In [ ]:
#TODO: devo estrarre TSNE??
# Come continuo ???